In [2]:
"""Simulation of the velocity distribution in a gas. """

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation

# box dimensions.
dim = 2

# Number of particles.
N = 200

# Simulation duration T and increment dt [s].
T = 10
dt = 0.01

# Smallest time difference at which collisions than simultaneously be accepted [s].
epsilon = 1e-9

# For each wall, the distance from the coordinate origin
# wall_d and an outward-pointing normal vector wall_n specified.
wall_d = np.array([2.0, 2.0, 2.0, 2.0])
wall_n = np.array([[0, -1.0], [0, 1.0], [-1.0, 0], [1.0, 0]])
print(f"wall_d = {wall_d}")
print(f"wall_n = {wall_n}")


# Randomly position the masses in the area
# x= -1.9 ... 1.9 m and y = -1.9 ... 1.9 m.
r0 = 1.9 * (2 * np.random.rand(N, dim) - 1)
print(f"r0 = {r0} ")


# Choose random speeds with magnitude 1 m/s.
v0 = -0.5 + np.random.rand(N, dim)
print(f"v0 = {v0} ")


v0 /= np.linalg.norm(v0, axis=1).reshape(-1, 1)
print(f"v0 = {v0} ")


# All particles get the same mass [kg].
radius = 0.05 * np.ones(N)
print(f"radius = {radius} ")


# All particles get the same mass [kg].
m = np.ones(N)
print(f"m = {m} ")


# Set the maximum for displaying the histogram
# Speed, the maximum number of particles per bar and the number of bars (n_bins) fixed.
v_max = 3.0
n_max = 50
n_bins = 15

# Create arrays for the simulation result.
t = np.arange(0, T, dt)
r = np.empty((t.size, N, dim))
v = np.empty((t.size, N, dim))
r[0] = r0
v[0] = v0


print(f"t ={t} ")

print(f"r ={r} ")

print(f"v ={v} ")

print(f"r0 ={r[0]} ")
print(f"v0 ={v[0]} ")

wall_d = [2. 2. 2. 2.]
wall_n = [[ 0. -1.]
 [ 0.  1.]
 [-1.  0.]
 [ 1.  0.]]
r0 = [[ 1.81482784 -1.2081487 ]
 [ 0.30369319 -1.88608782]
 [ 0.29839694 -1.00324241]
 [ 1.89321532  0.39141015]
 [-1.57284125  1.22040644]
 [-1.84041124 -1.72573839]
 [ 0.76153969  0.76657704]
 [-1.65159724  0.44539578]
 [ 1.00996186  1.40586199]
 [-1.18424631 -0.43644114]
 [-0.9182179   1.06526055]
 [ 0.41472504 -1.83365752]
 [ 0.76983401 -0.53301824]
 [ 1.17055196  1.54120814]
 [-0.64591192  0.33945745]
 [ 0.95701291  0.26021219]
 [ 0.66784871  0.02866671]
 [ 1.05869407 -1.0970902 ]
 [-0.93784302  1.63925931]
 [-1.41486835  0.64911891]
 [ 1.38699097  0.71112422]
 [ 1.33553735  0.58929736]
 [ 1.56296381 -1.28042436]
 [ 1.11262581  1.01808425]
 [ 1.1341811   0.05325831]
 [ 1.55522078 -1.06252003]
 [ 1.40990464  0.1489306 ]
 [ 0.40581334  1.53758334]
 [ 0.79213657 -0.79061818]
 [ 1.26578575 -1.37928584]
 [ 0.02464614  0.27446527]
 [ 0.09880568 -1.39843193]
 [ 0.92929195 -0.24615744]
 [ 0.27429696  1.68094163]


In [3]:

def collision_time(r, v):
    """Returns the time until the next particle collision and the indices of the participating particles. """

    # Create N x N x dim arrays that match the pairwise
    # Location and speed differences included:
    # dr[i, j] is the vector r[i] - r[j]
    # dv[i, j] is the vector v[i] - v[j]
    dr = r.reshape(N, 1, dim) - r
    dv = v.reshape(N, 1, dim) - v
    
    print(f"dr ={dr} ")

    print(f"dv ={dv} ")
    

    # Create an N x N array containing the squared absolute value of the
    # Contains vectors from array dv.
    dv2 = np.sum(dv * dv, axis=2)
    
    print(f"dv2 ={dv2} ")
    
    
    # Create an N x N array containing the pairwise sum
    # contains the radii of the particles.
    rad = radius + radius.reshape(N, 1)
    print(f"rad ={rad} ")
    
    # To determine the time of the collision,
    # form quadratic equation t² + 2 a t + b = 0 be resolved.
    # Only the smaller solution is relevant.
    a = np.sum(dv * dr, axis=2) / dv2
    b = (np.sum(dr * dr, axis=2) - rad ** 2) / dv2
    D = a**2 - b
    t = -a - np.sqrt(D)
    
    
    print(f"a = {a},   b = {b}, D = {D},   t = {t} ")
    

    # Find the smallest positive instant of a collision
    t[t <= 0] = np.NaN
    print(f"t[t <= 0] ={t[t <= 0]} ")
    
    
    t_min = np.nanmin(t)
    print(f"t_min ={t_min} ")
    
    
    # Find the corresponding particle indices.
    i, j = np.where(np.abs(t - t_min) < epsilon)
    print(f"i = {i},   j = {j} ")
    
    
    # Select the first half of the indices because each
    # Particle pairing occurs twice.
    i = i[0:i.size // 2]
    j = j[0:j.size // 2]
    print(f"i = {i}")
    print(f"j = {j} ")

    
    
    # Return time and particle indices. if no collision occurs, then return inf.
    if np.isnan(t_min):
        t_min = np.inf
        print(f"t_min = {t_min}")

    return t_min, i, j

In [ ]:
def collision_wall(r, v):
    """Returns the time until the next wall collision, the index of the particles and the index of the wall. """

    # Calculate the time of the collision of the particles # one of the walls.
    # The result is an N x number of walls - arrays.
    distance = wall_d - radius.reshape(-1, 1) - r @ wall_n.T
    print(f"distance = {distance}")
    
    t = distance / (v @ wall_n.T)
    print(f"t = {t}")
    

    # Ignore all non-positive tenses.
    t[t <= 0] = np.NaN
    print(f"t[t <= 0] = {t[t <= 0]}")
    
    
    # Ignore all times when the particle moves
    # against the normal vector. Actually
    # this shouldn't happen at all, but due to
    # rounding errors it can happen that a particle
    # is slightly outside a wall.
    t[(v @ wall_n.T) < 0] = np.NaN
    print(f"t[(v @ wall_n.T) < 0] = {t[(v @ wall_n.T) < 0]}")
    
    # Find the smallest point in time and give the time and indices back.
    t_min = np.nanmin(t)
    print(f"t_min = {t_min}")
    
    particle, wall = np.where(np.abs(t - t_min) < epsilon)
    print(f"particle = {particle}")
    print(f"wall = {wall}")
    
    
    return t_min, particle, wall


In [ ]:
# Calculate the time until the first collision and partners involved.
dt_particle, particle1, particle2 = collision_time(r[0], v[0])
print(f"dt_particle = {dt_particle}")
print(f"particle2 = {particle2}")

dt_wall, particle_w, wall = collision_wall(r[0], v[0])
print(f"d_wall = {d_wall}")
print(f"particle_w = {particle_w}")
print(f"wall = {wall}")

dt_collision = min(dt_particle, dt_wall)
print(f"wall = {dt_collision}")


In [ ]:
# Loop over the time steps.
for i in range(1, t.size):
    # Copy the positions from the previous time step.
    r[i] = r[i - 1]
    v[i] = v[i - 1]

    # Time that has already been simulated in this time step..
    t1 = 0

    # Handle all collisions in this one in turn timestep.
    while t1 + dt_collision <= dt:
        # Move all particles forward until collision.
        r[i] += v[i] * dt_collision

        # Collisions between particles among themselves.
        if dt_particle <= dt_wall:
            for k1, k2 in zip(particle1, particle2):
                dr = r[i, k1] - r[i, k2]
                dv = v[i, k1] - v[i, k2]
                er = dr / np.linalg.norm(dr)
                m1 = m[k1]
                m2 = m[k2]
                v1_s = v[i, k1] @ er
                v2_s = v[i, k2] @ er
                v1_s_neu = (2 * m2 * v2_s +
                            (m1 - m2) * v1_s) / (m1 + m2)
                v2_s_neu = (2 * m1 * v1_s +
                            (m2 - m1) * v2_s) / (m1 + m2)
                v[i, k1] += -v1_s * er + v1_s_neu * er
                v[i, k2] += -v2_s * er + v2_s_neu * er

        # Collisions between particles and walls.
        if dt_particle >= dt_wall:
            for n, w in zip(particle_w, wall):
                v1_s = v[i, n] @ wall_n[w]
                v[i, n] -= 2 * v1_s * wall_n[w]

        # Within this time step was a duration
        # dt_collision already covered.
        t1 += dt_collision

        # Since collisions have taken place, recalculate.
        dt_particle, particle1, particle2 = collision_time(r[i], v[i])
        dt_wall, teilchen_w, wand = collision_wall(r[i], v[i])
        dt_collision = min(dt_particle, dt_wall)

    # Now find until the end of the current time step (dt).
    # no more collision. We move all particles up
    # forward to the end of the time step and don't have to
    # Check for collisions again.
    r[i] = r[i] + v[i] * (dt - t1)
    dt_collision -= dt - t1

    # Give an information about the progress of the simulation in percent off.
    print(f'{100*i/t.size:.1f} %')

In [ ]:
# Create a figure.
fig = plt.figure(figsize=(8, 4))
fig.set_tight_layout(True)

# Create an Axes for the animation of the movement of the particles.
ax1 = fig.add_subplot(1, 2, 1)
ax1.set_title('particle motion')
ax1.set_xlabel('x [m]')
ax1.set_ylabel('y [m]')
ax1.set_xlim([-2.1, 2.1])
ax1.set_ylim([-2.1, 2.1])
ax1.set_aspect('equal')
ax1.grid()

In [ ]:
# Create a circle for each particle.
circles = []
for i in range(N):
    c = mpl.patches.Circle([0, 0], radius[i])
    ax1.add_artist(c)
    circles.append(c)

In [ ]:
# Create a second axis for the histogram.
ax2 = fig.add_subplot(1, 2, 2)
ax2.set_title('speed distribution')
ax2.set_xlabel('|v| [m/s]')
ax2.set_ylabel('Anzahl der Teilchen')
ax2.set_ylim([0, n_max])
ax2.grid()

In [ ]:

# Generate the data for the histogram.
hist, bins = np.histogram(np.linalg.norm(v[0], axis=1),
                          bins=n_bins, range=[0, v_max])


In [ ]:
# Display the histogram as a bar chart.
bar = ax2.bar(bins[:-1], hist, width=v_max / n_bins,
                 align='edge', edgecolor='white', zorder=2)


In [ ]:

def update(n):
    # Update the positions of the particles.
    for i, k in enumerate(circles):
        k.set_center(r[n, i])

    # Calculate the histogram for the current time step.
    hist, bins = np.histogram(np.linalg.norm(v[n], axis=1),
                              bins=n_bins, range=[0, v_max])

    # Update histogram bars.
    for i, p in enumerate(bar):
        p.set_height(hist[i])

    return circles + list(bar)


In [ ]:

# Create the animation and start it.
ani = mpl.animation.FuncAnimation(fig, update, interval=30,
                                  frames=t.size, blit=True)
plt.show()